In [2]:
# libraries
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox

from PIL import ImageTk, Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

from keras.utils.np_utils import to_categorical 
from keras.models import load_model
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv("data\HAM10000_metadata.csv")
df

In [ ]:
df.info()

#### Note:
Dataframein(**df**) içerisinde **dx** feature(column)'unu countplot yapılıyor. Yani içerisinde bulunan kanser hücresinden kaç tane olduğunu ve dağılımını grafikle bize gösteriyor.
- **%matplotlib qt** eklentisinin düzgün çalışması(değerlerin doğru görüntülenebilmesi) için figure'un tam ekran yapılması gerekmektedir!

In [ ]:
%matplotlib qt5 
# %matplotlib widget # embedded
sns.countplot(x = "dx", data = df)

## 1-Data Preprocessing

- Öncelikle image'lerin dosya konumları(path) dataFrame'e path adıyla sütun olarak eklenir.

In [ ]:
img_folder_name = 'img/'
extension = ".jpg"
#sample image path = img_folder_name + image_id[i] + extension
# ismi path olan, dosya yoluna eşit değerleri olan yeni column oluşturulur.
df['path'] = [ img_folder_name + i + extension for i in df['image_id'] ]
df

- df['path']'in içerisindeki her bir satırı fonksiyon yapması için map kullanılır. 
- x = 'path' e karşılık gelir. 
- x pathindeki resmi aç diyoruz. 
- Ancak bu resmi biz array olarak tanımlamak ve tutmakistiyoruz, bu yüzden np.asarray cast'ı ekleriz.

- Bu işlem birkaç dakika süreceği için aşağıda ```df.to_pickle("df.pkl")``` işlemini yaparak, verilerin son halini df.pkl dosyasında tutmuş oluruz.

In [ ]:
df["image"] = df["path"].map( lambda x: np.asarray(Image.open(x).resize((100,75))))

In [ ]:
df.head(5)

In [ ]:
%matplotlib inline
plt.imshow(df["image"][0]) # imshow: bir matrisi image şeklinde gösterir.

In [ ]:
# string değerleri numeric değerlere kodlar(convert).
df["dx_idx"] = pd.Categorical(df["dx"]).codes

In [ ]:
df.head(3)

In [ ]:
df.to_pickle("df.pkl") # Yukarıda yaptığımız işlemleri her seferinde yapmak zahmetli. Verimizi convert edeceğimiz, verimizi depolayan bir pickle dosyası olduğunu bilmek yeterli.

## 2-Read Pickle File

In [3]:
df = pd.read_pickle("df.pkl") # verilerimizi hızlıca yukarıdaki adımları yapmaya gerek kalmadan bu bloku debug ederiz, dataFrame nesnesine dönüştürürek.
df.head(3)

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,image,dx_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,img/ISIC_0027419.jpg,"[[[190, 153, 194], [192, 154, 196], [191, 153,...",2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,img/ISIC_0025030.jpg,"[[[23, 13, 22], [24, 14, 24], [25, 14, 28], [3...",2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,img/ISIC_0026769.jpg,"[[[185, 127, 137], [189, 133, 147], [194, 136,...",2


## 3-Standardization

- df['image'] sütununu yazdırdığımızda indeksleriyle birlikte bir array görünümlü ama indeksi de olan bir tür olarak görülecektir. Bu yüzden ilk önce list sonrasında da array'e çevirerek tüm indekslerden kurtulmuş oluruz. 

- **x_train** burada bizim eğitilecek verilerimiz image dosyalarıdır. Bu image dosyaları üzerinde algoritmalar ve modeller uygulanacaktır. Ve daha sonra elimizde olan görüntüler bu x_train verileriyle karşılaştırılacaktır.
- **x_test** görürsek bunun anlamı, eğitilmiş x_train verilerimizle, bunlardan ayrı hiç eğitilmemiş **yeni verilerimizi**, **x_train** verilerimize göre karşılaştırılacak ve yeni verilerimizin **target class** dediğimiz kanser türü (dx) sonuçları oluşturulacak.

In [4]:
x_train = np.asarray(df["image"].tolist())

In [5]:
#  stardardization
x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)
x_train = (x_train - x_train_mean)/x_train_std

- **y_train** bizim target classlarımızdır. Yani outputumuz olan kanser hücresi türüdür. 
- Bu veri image dosyalarının olduğu verilerden(**x_train**) ayrı hesaplanacağı için ayrı tutulur.

- Aşağıda **One Hot Encoding** yöntemi kullanılmıştır. Bu yöntemdeki mantık:
**dx_idx** sütunu içerisinde 7 farklı sonuca(kanser hücresi türüne) karşılık gelen sayı vardır, bunlar: 0, 1, 2, 3, 4, 5, 6
- **One Hot Encoding** yöntemiyle:
> <br>
> 0 yerine 0000000 => herbiri 7 adet rakamdan oluşur. (num_classes = 7)<br> 
> 1 yerine 0100000 <br>
> 2 yerine 0010000 <br>
> 3 yerine 0001000 <br>
> 4 yerine 0000100 <br>
> 5 yerine 0000010 <br>
> 6 yerine 0000001 yazılır.<br><br> 

In [6]:
# one hot encoding
target_class_count = len(pd.unique(df['dx_idx']))
y_train = to_categorical(df["dx_idx"], num_classes = target_class_count)

In [7]:
print("x_train: " , x_train.shape)
print("y_train: " , y_train.shape)

x_train:  (10015, 75, 100, 3)
y_train:  (10015, 7)


# 4- Building the Model => Bir Deep Learning Algoritması: CNN

In [8]:
x_train.shape # 10015 instance, 75x100, 3 (RGB color image)
n_row = x_train.shape[1]
n_col = x_train.shape[2]
col_type = x_train.shape[3]
print(n_row, n_col, col_type)

75 100 3


In [9]:
input_shape = (n_row, n_col, col_type)
target_class_count

7

In [10]:
model = Sequential() # Sırayla tüm CNN layerlerımızı adım adım eklememiz için gerekli.

## LAYER 1: Convolutional Layer
> <br>
> Resmin özelliklerini saptamak için kullanılır.
> <br> <br>
- Bu katman CNN’nin ana yapı taşıdır. 
- Resmin özelliklerini algılamaktan sorumludur. 
- Bu katman, görüntüdeki düşük ve yüksek seviyeli özellikleri çıkarmak için resme bazı fitreler uygular. 
- Örneğin, bu filtre kenarları algılayacak bir filtre olabilir. Bu filtreler genellikle çok boyutludur ve piksel değerleri içerirler.
- (5x5x3) özelliğinde bir matris için 5 matrisin yükseklik ve genişliğini, 3 matrisin derinliğini temsil eder.

```python
Conv2D(32, kernel_size = (3,3), activation = "relu", padding = "Same", input_shape = input_shape)
```
kodunu inceleyecek olursak,
- 32:neron sayısı, 
- kernel_size = (3,3): convolutional filtre boyutu 3x3 matris, 

- activation = "relu": Bu bir ReLu activation function'udur. Genellikle CNN de kullanılır. Ana avantajı aynı anda tüm nöronları aktive etmemesidir. Yani bir nöron negatif değer üretirse, aktive edilmeyeceği anlamına gelir. Negatif değerler üreten nöronlar sıfır değerini alır. Bu durum, **ReLU**’nun Hiperbolik Tanjant ve Sigmoid fonksiyonundan daha verimli ve hızlı çalışmasını sağlar. Bu nedenle ReLU, çok katmanlı sinir ağlarında daha çok tercih edilir. 

- padding = "Same": Veri kaybını önlemek için
- input_shape = input_shape: verinin boyutu ve rgb özelliği olması.

#### Özetle
1. **Convolution input** resmin(matrisin) üzerinde **convolutional filter**’ın kaydırılması sonucu yapılır. 

2. Çakışan sayılar çarpılır. 
3. Çarpım sonucu elde edilen sayılar toplanarak **feature map** (output) matrisine aktarılır. 
4. Bu işlem input resminin boyutunu düşürürken veri kaybına da sebep oluyor. Veri kaybını önlemek için **same padding** metodu kullanılıyor. **Same padding** input resminin etrafına sıfır değerlerinden oluşan bir **çerçeve** eklenmesine deniyor. Bu sayede veri kaybı önleniyor. 
5. **Feature map** resme ait bir **feature** tutar, örneğin input arabaysa, bir feature map arabanın farını tutabilir, fakat arabanın diğer tüm özellikleri yani kapısı, camı vs gibi tüm feature’ları tespit edebilmek için çok sayıda **feature map**’e ihtiyaç duyarız. 
6. **Convolution** işleminin ardından feature map’e bir **aktivasyon fonksiyonu** olan **reLu** uygulanır. İşlemin **reLu** fonksiyonuna sokulmasının sebebi eksi değerlerin(nöronların) sıfırlanmasıdır. 
7. İkinci kez bu **Convolutional Layer**'ı tekrar ekleriz. Bunun sebebi tecrübelere dayalıdır. Ancak esas sebep: İlk filtreyi uyguladığımızda, bir **Feature Map** oluşturuyor ve bir **özellik** türünü tespit ediyoruz. Ardından, ikinci bir filtre kullanıp başka bir **özellik** türünü algılayan ikinci bir **Feature Map** oluştururuz.
8. İkinci kez CNL uygularken, input_shape yazmıyorum. Çünkü modelimi Sequential() bir yapıda kurdum. Sequential(feature1, feature2) => yani ilk baştaki işlemin sonucunda oluşan feature_map, artık 2. işlemin inputu olacak. Ancak ilk baştakinin inputunu mecbur belirtmem gerekir, 2. işlem gibi şanslı değil.

In [11]:
model.add(Conv2D(32, kernel_size = (3,3), activation = "relu", padding = "Same", input_shape = input_shape))
model.add(Conv2D(32, kernel_size = (3,3), activation = "relu", padding = "Same"))

## LAYER 2: Pooling Layer
1. Bu katman, CovNet’teki ardışık **convolutional katmanları** arasına sıklıkla eklenen bir katmandır.

2. Bu katmanın görevi, gösterimin kayma boyutunu ve network içindeki parametreleri ve hesaplama sayısını azaltmak içindir. Bu sayede ağdaki uyumsuzluk kontrol edilmiş olur. Bunun dışında **overfitting** (aşırı öğrenme) sorununu çözmek için de kullanılır.
3. Birçok Pooling işlemleri vardır, fakat en popüleri **max pooling**’dir. 
4. Yine aynı prensipte çalışan **average pooling**, ve **L2-norm pooling** algoritmaları da vardır.
5. Feature map matrisi üzerinde belirlenen ölçülerde bir pencere dolaştırılır. Pencere içerisinde ki max değerin alınmasıyla yapılan bir işlemdir. Bu sayede, sinir ağının doğru karar vermesi için için yeterli bilgiyi içeren daha küçük çıktıları kullanmış olur.

In [12]:
model.add(MaxPool2D(pool_size = (2,2)))

In [13]:
model.add(Dropout(0.25)) # overfitting'i önlemek için dropout ekleriz.

## LAYER 3: Tekrardan, Yine => Convolutional Layer
- Nöron sayısını bu sefer 32 değil de 64 yaptık.

In [14]:
model.add(Conv2D(64, kernel_size = (3,3), activation = "relu", padding = "Same", input_shape = input_shape))
model.add(Conv2D(64, kernel_size = (3,3), activation = "relu", padding = "Same"))

## LAYER 4: Tekrardan Pooling Layer


In [15]:
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.5)) # overfitting'i önlemek için dropout ekleriz.

 ## Layer 5: Flattening Layer
 1. Genel olarak, sinir ağları(neural network), input verilerini tek boyutlu bir diziden alır.
 
 2. **Convolution** ve **Pooling** işlemlerinden sonra ortaya çıkan matrisleri **n satır 1 sütundan** oluşan vektörlere dönüştürme işlemine **flattening** denmektedir. Örneğin: 2x2 => 4x1 ya da 3x3 => 9x1
 3. Bu vektörler, son ve en önemli katman olan  **artificial neural network**’ün olduğu kısım olan **Fully Connected Layer**ın inputları olacaktır.

In [16]:
 model.add(Flatten())

 ## Layer 6: Full Connected/Dense Layer
- Girdi resmimizi sinir ağları ile eğitebileceğimiz kıvama getirildikten sonra (Yani matris halinde olan görselimiz düz bir vektör haline getirildikten sonra) geriye sadece klasik sinir ağlarındaki çalışma mantığı kalıyor. 

- Yine katmanlardaki nodelarda (düğüm) özellikler tutuluyor ve weight (ağırlık) ve bias değiştirilerek öğrenme sürecine giriliyor.

In [17]:
model.add(Dense(128,activation="relu")) # nöron sayısı ne kadar fazla olurs o kadar iyi ancak veri kapasitesi artacak.
model.add(Dropout(0.5)) # overfitting'i önlemek için dropout ekleriz.

 ## Layer 7: Output Layer
- Output layer yine dense olacak(ANN).

- Dikkat edilmesi nokta, outputta 7 farklı sonuçtan 1 tanesinin çıkabilmesi için burada **nöron sayısı** yerine **target class** sayısını gireriz.
- Activation fonksiyonu olarak da **reLu** yerine **softmax** kullanılacaktır.
- **Softmax** activasyonu **Target Class** sayısı **2**'den fazla ise kullanılan bir yöntemdir. (Bizim veri setinde **7 target class** var.)
- Bu işlem sonucunda şu şekilde bir output dizisi ortaya çıkar:
0.1 - 0.2 - 0.3 - 0.05 - 0.25 - 0.02 - 0.08
- Yukarıdaki 7 tane elemanı olan target sınıfında en büyük değer 0.3'tür. Yani Yüzde 30 olasılığıyla sonuç 3. sıradaki classtır diyebiliriz.

In [18]:
model.add(Dense(target_class_count, activation="softmax"))
model.summary() # özetleyeceğimiz kısım.

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 75, 100, 32)       896       
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 100, 32)       9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 37, 50, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 37, 50, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 37, 50, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 37, 50, 64)        36928     
                                                        

## 5-Compiling the Model

- Modelin derlenmesi **model.compile()** fonksiyonu ile yapılır. 

- Bu fonksiyon **optimizer**, **loss**(kayıp)  ve **metrics** (metrik) olmak üzere üç temel parametre alır. Farklı parametreler de kullanılabilir.
1. **Optimizer**, öğrenme oranını(**learning rate**) kontrol eder. Optimizer olarak **“**Adam** kullanıyoruz. **Adam** genellikle birçok durumda kullanmak için iyi bir **optimizasyon** algoritmasıdır. **Adam** algoritması, training boyunca **learning rate**'i ayarlar.

2. **Learning Rate** (Öğrenme oranı), model için optimal ağırlıkların ne kadar hızlı hesaplandığını belirler. Daha küçük bir **learning rate**, daha kesin ve iyi ağırlıklara (belirli bir noktaya kadar) yol açabilir, bu modelin daha **iyi** öğrenmesi anlamına gelir ancak ağırlıkların hesaplanması için gereken süre daha **uzun** olacağından dolayı eğitim(train) süresi uzayacaktır.
3. **Loss** fonksiyonumu,training aşamasında giderek azalmasını bekleriz. Bu fonksiyon için **categorical_crossentropy** kullanacağız. Bu, sınıflandırma problemleri için en yaygın kullanılan fonksiyon türüdür.
4. **Accuracy** de loss ile ters orantılı olarak artmalıdır. Bu model tahminleri doğru yapabiliyor anlamına gelir.
5. Eğitim(Train) sırasında modelin nasıl bir performans gösterdiğini yorumlamak için her bir **epoch** sonunda **validation(test)** seti ile elde edilen **doğruluk** ve **loss** miktarını görmek için **accuracy** metriğini kullanırız.


In [19]:
optimizer = Adam(learning_rate = 0.001) # The 'lr' argument is deprecated, so we use learnign_rate.
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

## 6-Training the Model

1. **fit** fonksiyonu içerisine (**validation_data**=(X_test, y_test)) da yazılırdı. Bu projede test işlemi yapılmadığı için eklenmedi.

2. **Batch Size**: Batch sayısı modelin eğitilmesi aşamasında aynı anda kaç adet verinin işleneceği anlamına gelir. Ne kadar fazla yapılırsa RAM'e o kadar yük biner, ama işlem daha kısa sürer! :) Bu sayı genellikle 2'nin üssü olması yani 2,4,8,16,32 gibi sayıların olması tavsiye edilir.
3. **Epoch**: 
- verilerin kaç kez **train** edeceğini söyler. Mesela 10000 adet resmimiz var. Epoch 2 ise tüm 10000 resim 2 kez train edilecektir.

- bu değerin küçük bir sayı olması, eğitim süresini kısa tutarken modelin performansı tam gelişmemiş olabilirken, büyük bir sayı olması ise eğitim süresi çok uzun olur ve model gelişimini çoktan tamamlamış olabilir. Yani gereksiz eğitim yapılmış olabilir optimumm noktayı bulmakta zorlanabiliriz
- Bunun için en iyi yöntem **büyük bir sayı** belirlenmesi ve modelin gelişimini tamamladığı noktada eğitimi durdurmaktır, buna **early stopping** deniyor. 
- Bu yöntem bu projede kullanılmadı ancak bu yöntem araştırılacak.
- Accuracy değeri her epoch değerinde artıyor. Bu değerin durduğu epoch değerini gözlemek gerekir.
4. **Shuffle**: Her **Epoch**ta, verilerin yerlerinin değiştirilmesidir. Eğitimin daha doğru olması için kullanılır. True ve False boolean değerleri alır.

5. **Verbose**: 0, 1 ve 2 değerlerinden birisini alır. 
- 0 değeri, eğitim sırasında ekranda bir sonuç göstermezken, 
- 1 değeri, **progres bar** gibi anlık olarak güncellenen sonçları gösterir. 
- 2 değeri ise her bir epoch sonunda tek bir satır olarak çıktı verir.

In [20]:
history = model.fit(x = x_train,
                    y = y_train,
                    batch_size = 64, 
                    epochs = 5,
                    verbose = 1,
                    shuffle = True)

Epoch 1/5
157/157 [==============================] - 205s 1s/step - loss: 1.0580 - accuracy: 0.6643
Epoch 2/5
157/157 [==============================] - 212s 1s/step - loss: 0.8772 - accuracy: 0.6825
Epoch 3/5
157/157 [==============================] - 218s 1s/step - loss: 0.8317 - accuracy: 0.6966
Epoch 4/5
157/157 [==============================] - 180s 1s/step - loss: 0.7975 - accuracy: 0.7055
Epoch 5/5
157/157 [==============================] - 177s 1s/step - loss: 0.7636 - accuracy: 0.7185


### Note
##### Modelin performansını arttırmak için modelin ayarları ile oynamak, daha fazla katman veya nöron eklemek, verileri çoğaltmak veya daha güzel veriler seçmek gerekebilir.

- Modeli train etme işlemi epoch değerine ve learning rate değerlerine göre uzun sürede gerçekleşebilir.

- Bu yüzden mevcut modeli kaybetmek istemeyiz ve modeli bir yerde **my_model1.h5** adıyla kaydedebiliriz. 
- Özetle modelde learning rate ve epoch değerleriyle oynayarak yeni isimler verilebilir ve sonuçlar karşılaştırılabilir. Farklı seçenekleri kullanıcı arayüzünde gösterilecek Model1 veya Model2 diye.

In [21]:
model.save("my_model1.h5")
#model.save("my_model2.h5")

In [22]:
# load models
model1 = load_model("my_model1.h5")
#model2 = load_model("my_model2.h5")

## 7-Using our models to make predictions (Yeni veriler ile tahmin yapmak)

- Modellerimizi yeni verilerle ilgili tahminlerde bulunmak için kullanmak istiyorsak, **model.predict()** fonksiyonunu kullanmamız gerekir. 

- Input olarak **test** verilerinden istediğimiz bir tanesini seçip verebiliriz.

- Ben **5. satırda** bulunan veriyi alıp **tahminde** bulunmasını istedim. 

- Sonuç olarak %60 oranında meningioma tümörü çıktı. Modelin başarı oranına göre gayet başarılı bir sonuş olduğunu söyeyebiliriz.

In [50]:
index = 1304
y_train[index] # 5. indexin gerçekteki değeri, şimdi aşağıda tahminleme yapalım.

array([0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [51]:
y_pred = model1.predict(x_train[index].reshape(1,75,100,3))
print(y_pred)
print("Maximum value: ",np.max(y_pred)) # Tüm olasılıkların toplamının 1 olmasını bekleriz.
print("Sum: ", np.sum(y_pred)) # Tüm olasılıkların toplamının 1 olmasını bekleriz.

[[0.00262048 0.00204493 0.30294672 0.00124833 0.40381494 0.28689888
  0.00042572]]
Maximum value:  0.40381494
Sum:  1.0


In [52]:
# class_names = ['', '', '']
print("Predict result:", np.argmax(y_pred))
print("Reel result:", np.argmax(y_train[index]))

Predict result: 4
Reel result: 4
